In [2]:
import sys
import os

# Define the path to your folder
folder_path = f'..\\..\\..\\Models\\AutoDiffusion' 

# Add the folder to sys.path
sys.path.append(folder_path)

In [3]:
import numpy as np
import process_edited as pce
import process_GQ as pce
import autoencoder as ae
import diffusion as diff
import TabDDPMdiff as TabDiff
import pandas as pd
import torch
import os
import time
import numpy as np
from ForestDiffusion import ForestDiffusionModel

In [6]:
strings_set = ['mammography']
Model = 'AutoDiff'
metrics_list = []
for dataset in strings_set:
    print(f"Result Metrics for AutoDiff Autoencoder & ForestDIffusion for {dataset} dataset")
    file_path = f'..\\..\\..\\Datasets\\Original Data\\{dataset}.csv'
    # Read dataframe
    # print(file_path)
    real_df = pd.read_csv(file_path)
    #real_df = real_df.drop('url', axis=1)
    # # Step 2: Inspect the data and check for class imbalance
    # # Assuming the last column is the label, and the rest are features
    X = real_df.iloc[:, :-1].values  # Features
    y = real_df.iloc[:, -1].values  # Labels (binary classification)
    #  # Separate the minority class
    # Find the minority class
    
    real_minortiy = real_df[y == 1]

    threshold = 0.01 # Threshold for mixed-type variables
    parser = pce.DataFrameParser().fit(real_minortiy, threshold)
    ################################################################################################################
    # Auto-encoder hyper-parameters
    device = 'cuda' #@param {'type':'string'}
    n_epochs = 10 #@param {'type':'integer'}
    eps = 1e-5 #@param {type:"number"}
    weight_decay = 1e-6 #@param {'type':'number'}
    maximum_learning_rate = 1e-2 #@param {'type':'number'}
    lr = 2e-4 #@param {'type':'number'}
    hidden_size = 250
    num_layers = 3
    batch_size = real_minortiy.shape[0] # Full batch

    ds = ae.train_autoencoder(real_minortiy, hidden_size, num_layers, lr, weight_decay, n_epochs, batch_size, threshold)
    latent_features = ds[1].detach()

    from ForestDiffusion import ForestDiffusionModel


    # Convert to NumPy array
    array = latent_features.detach().cpu().numpy()
    forest_model = ForestDiffusionModel(array, label_y=None, n_t=50, duplicate_K=100, bin_indexes=[], cat_indexes=[], int_indexes=[], diffusion_type='flow', n_jobs=-1)
    minority_fake = forest_model.generate(batch_size=len(real_minortiy)) # Adjust the batch size to create a balanced dataset
    sample=torch.tensor(minority_fake, dtype=torch.float32)
    sample.shape
    gen_output = ds[0](sample, ds[2], ds[3])
    gen_df = pce.convert_to_table(real_minortiy, gen_output, threshold)

    output_directory =  f'..\\..\\..\\Datasets\\Synthetic Data\\'
    filename = f'{Model}+Forest_{dataset}_Synthetic.csv'
    output_file = os.path.join(output_directory, filename)
    gen_df.to_csv(output_file, index=False) 


    # Select a random sample of the generated data
    selected_samples = gen_df.sample(n=min(100,gen_df.shape[0]), random_state=42)  # For reproducibility
    # Syn _df will be the dataset after augmentation
    syn_df = pd.concat([real_df, selected_samples], ignore_index=True)

Result Metrics for AutoDiff Autoencoder & ForestDIffusion for mammography dataset


  0%|          | 0/10 [00:00<?, ?it/s]

In [7]:
print(type(array))
print(array.shape)
print(real_df.shape)
print(type(minority_fake))

<class 'numpy.ndarray'>
(260, 13)
(11183, 7)
<class 'numpy.ndarray'>


In [4]:
# strings_set = ['diabetes','oil','yeast_ml8_dataset','creditcard_sampled','HTRU','mammography']
strings_set = ['mammography']
Model = 'AutoDiff'
metrics_list = []


for dataset in strings_set:
    print(f"Result Metrics for AutoDiff Autoencoder & ForestDIffusion for {dataset} dataset")
    file_path = f'..\\..\\..\\Datasets\\Original Data\\{dataset}.csv'
    # Read dataframe
    # print(file_path)
    real_df = pd.read_csv(file_path)
    #real_df = real_df.drop('url', axis=1)
    # # Step 2: Inspect the data and check for class imbalance
    # # Assuming the last column is the label, and the rest are features
    X = real_df.iloc[:, :-1].values
    X_minority=real_df[y==1].iloc[:, :-1].values# Features
    y = real_df.iloc[:, -1].values
    Y_minority=real_df[y==1].iloc[:, :-1].values# Labels (binary classification)
    #  # Separate the minority class
    # Find the minority class
    
   

    threshold = 0.01 # Threshold for mixed-type variables
    parser = pce.DataFrameParser().fit(real_minortiy, threshold)
    ################################################################################################################
    # Auto-encoder hyper-parameters
    device = 'cuda' #@param {'type':'string'}
    n_epochs = 10 #@param {'type':'integer'}
    eps = 1e-5 #@param {type:"number"}
    weight_decay = 1e-6 #@param {'type':'number'}
    maximum_learning_rate = 1e-2 #@param {'type':'number'}
    lr = 2e-4 #@param {'type':'number'}
    hidden_size = 250
    num_layers = 3
    batch_size = real_minortiy.shape[0] # Full batch

    ds_X_minority = ae.train_autoencoder(X_minority, hidden_size, num_layers, lr, weight_decay, n_epochs, batch_size, threshold)
    latent_features = ds[1].detach()

    # Convert to NumPy array
    array = latent_features.detach().cpu().numpy()
    forest_model = ForestDiffusionModel(array, label_y=None, n_t=50, duplicate_K=100, bin_indexes=[], cat_indexes=[], int_indexes=[], diffusion_type='flow', n_jobs=-1)
    minority_fake = forest_model.generate(batch_size=len(real_minortiy)) # Adjust the batch size to create a balanced dataset
    # sample=torch.tensor(minority_fake, dtype=torch.float32)
    # sample.shape
    # gen_output = ds[0](sample, ds[2], ds[3])
    # gen_df = pce.convert_to_table(real_minortiy, gen_output, threshold)



    # augmented_output_directory =  f'..\\..\\..\\Datasets\\Augmented Data\\'
    # filename = f'{Model}+Forest_{dataset}_Augmented.csv'
    # augmented_output_file = os.path.join(augmented_output_directory, filename)
    # syn_df.to_csv(augmented_output_file, index=False) 

    


Result Metrics for AutoDiff Autoencoder & ForestDIffusion for mammography dataset


NameError: name 'y' is not defined

In [10]:
print(type(array))
print(array.shape)
# print(real_df.shape)
print(minority_fake.shape)


<class 'numpy.ndarray'>
(260, 13)
(260, 13)


In [8]:
[print(array)]

[[ 0.8476851   0.6754753   1.1448116  ... -0.61594456 -1.0673927
  -0.7629952 ]
 [ 0.6765012   0.4272185  -0.22333136 ... -1.0574458  -1.107088
   0.02813436]
 [ 0.77710074  0.10258555  0.78371805 ... -1.2053951  -0.1259919
  -1.2822082 ]
 ...
 [-0.0473737   0.08445081  0.4095962  ... -0.56679124 -0.00832558
  -0.4477823 ]
 [ 0.40814197  0.5695954   0.09218815 ... -1.2956736  -0.35899812
  -0.7206599 ]
 [ 0.39317212  0.612952   -0.41354346 ... -1.4080838  -1.0621437
  -0.40539533]]


[None]

In [9]:
df=pd.DataFrame(array)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,3.077980,-15.055771,-12.541447,-8.873666,-4.389857,7.341290,-10.188528,7.184702,-14.179997,7.638948,7.483530,-8.648089,4.066612
1,-2.161444,-24.312395,-14.273751,-10.290889,-2.314354,8.544633,-6.862887,6.513667,-20.340391,3.810999,17.796349,-9.882925,1.272065
2,-3.442517,-4.512757,-5.885982,-0.427331,-5.220022,-4.780949,-4.378103,13.856998,-3.597708,9.397711,-6.992277,-5.273534,11.595331
3,-2.622749,-47.598675,-8.035440,-4.906667,-9.887181,16.637634,8.108945,6.682512,-5.649014,4.487415,9.396755,-3.408261,11.982435
4,-3.068099,-8.651458,-7.440869,-1.279265,-8.687286,-5.127207,-8.518230,13.493593,-3.051555,11.790753,0.275342,-1.655885,6.305118
...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,1.245036,-38.705284,-4.749161,-5.972995,-9.175056,8.476085,5.486655,12.370555,-3.316094,1.311978,1.307320,-2.815952,12.672902
264,-6.881784,-14.146931,-9.094768,-12.008457,3.567058,5.734353,-0.568215,14.993578,-2.334203,-1.499555,0.053442,-6.648342,11.487378
265,-0.482998,-59.255409,-12.081550,-16.564106,-6.138309,19.849867,4.292315,9.117833,-15.240891,-5.854153,17.994995,-6.251715,6.015072
266,3.322240,-15.822894,-14.267989,-9.964271,-5.580717,4.163420,-15.139224,3.119954,-19.759991,8.025823,11.851574,-11.476839,-1.463071


In [ ]:
import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
    import numpy as np
    import matplotlib.pyplot as plt
    from ForestDiffusion import ForestDiffusionModel
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import recall_score, f1_score


    X = real_df.iloc[:, :-1].values  # Features
    y = real_df.iloc[:, -1].values 
    # Check and print the original class distribution
    unique, counts = np.unique(y, return_counts=True)
    class_dist_before = dict(zip(unique, counts))
    print(f"Class distribution before augmentation: {class_dist_before}")# Labels (binary classification)

    X_balanced = augmented_df.iloc[:, :-1].values  # Features
    y_balanced = augmented_df.iloc[:, -1].values  # Labels (binary classification)

    # Check and print the Augmented class distribution
    unique, counts = np.unique(y_balanced, return_counts=True)
    class_dist_after = dict(zip(unique, counts))
    print(f"Class distribution after augmentation: {class_dist_after}")

    # Step 6: Split the dataset into training and test sets (original and balanced)
    X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X, y, test_size=0.3, random_state=42)
    X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(X_balanced, y_balanced, test_size=0.3, random_state=42)

    # Step 7: Train a simple classifier on both original and generated datasets
    clf_orig = RandomForestClassifier(random_state=42)
    clf_orig.fit(X_train_orig, y_train_orig)

    clf_bal = RandomForestClassifier(random_state=42)
    clf_bal.fit(X_train_bal, y_train_bal)

    # Step 8: Predict and calculate recall and F1 scores
    y_pred_orig = clf_orig.predict(X_test_orig)
    y_pred_bal = clf_bal.predict(X_test_orig)

    prec_orig = precision_score(y_test_orig, y_pred_orig)
    prec_bal = precision_score(y_test_orig, y_pred_bal)
    
    recall_orig = recall_score(y_test_orig, y_pred_orig)
    recalls_bal = recall_score(y_test_orig, y_pred_bal)

    f1_orig = f1_score(y_test_orig, y_pred_orig)
    f1_bal = f1_score(y_test_orig, y_pred_bal)

    # Step 9: Print and store the performance metrics
    # Store metrics in a dictionary
    metrics = {
    "Dataset": dataset,
    "Precision_Original": prec_orig,
    "Precision_Generated": prec_bal,
    "Recall_Original": recall_orig,
    "Recall_Generated": recalls_bal,
    "F1_Original": f1_orig,   
    "F1_Generated": f1_bal,
    "Num_Fake_Samples": len(augmented_df) - len(real_df),
    "Synthetic/Original_Ratio": 100*(len(augmented_df) - len(real_df))/len(real_minortiy)
    }

    # Append the dictionary to the list
    metrics_list.append(metrics)

    print(f"Precision score (original data): {prec_orig:.4f}")
    print(f"Precision score (generated data): {prec_bal:.4f}")
    print(f"Recall score (original data): {recall_orig:.4f}")
    print(f"Recall score (generated data): {recalls_bal:.4f}")
    print(f"F1 score (original data): {f1_orig:.4f}")
    print(f"F1 score (generated data): {f1_bal:.4f}")
    print("Classification Report (original data):\n", classification_report(y_test_orig, y_pred_orig))
    print("Classification Report (generated data):\n", classification_report(y_test_orig, y_pred_bal))


    print(f"Number of fake samples generated: {len(augmented_df)-len(real_df)}")
# Convert the list of dictionaries into a DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Save the DataFrame to a CSV file
metrics_df.to_csv("Auto_Diff_Forest_different_datasets_metric.csv", index=False)

# Print the DataFrame
print(metrics_df)

In [3]:
import pandas as pd
metrics_df=pd.read_csv("Auto_Diff_Forest_different_datasets_metric.csv")
metrics_df.head(6)

,Dataset,Precision_Original,Recall_Original,F1_Original,Precision_Generated,Recall_Generated,F1_Generated,Num_Fake_Samples
0,yeast_ml8_dataset,0.000000,0.000000,0.000000,1.000000,0.551020,0.710526,100
1,oil,0.600000,0.272727,0.375000,0.875000,0.636364,0.736842,41
2,mammography,0.916667,0.594595,0.721311,0.966102,0.770270,0.857143,100
3,HTRU,0.937644,0.835391,0.883569,0.979358,0.878601,0.926247,100
4,diabetes,0.638554,0.662500,0.650307,0.720430,0.837500,0.774566,100
5,creditcard_sampled,0.866667,0.764706,0.812500,0.875000,0.823529,0.848485,50


In [6]:
metrics_df[metrics_df['Dataset']=='creditcard_sampled'] 

,Dataset,Precision_Original,Recall_Original,F1_Original,Precision_Generated,Recall_Generated,F1_Generated,Num_Fake_Samples
5,creditcard_sampled,0.866667,0.764706,0.8125,0.875,0.823529,0.848485,50
